In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
287342,2.161301e+09,2,216,1,21613014,4,1,C,1.0,1.0,1,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
102657,1.191001e+09,1,119,1,11910007,9,2,C,1.0,1.0,1,8.0,2.0,NaN,NaN,NaN,NaN,13.0,NaN
38329,1.091102e+09,1,109,1,10911018,11,10,C,NaN,NaN,1,1.0,1.0,2.0,3.0,1.0,NaN,NaN,NaN
64066,1.122001e+09,1,112,2,11220009,1,6,C,1.0,1.0,1,10.0,1.0,11.0,1.0,1.0,NaN,NaN,NaN
326742,3.092000e+09,3,309,2,30920003,4,1,C,2.0,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [3]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [4]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [5]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
296310,2.152003e+09,sindh,thaeparkar,rural,21520028,B,1,male,present,head of household,1955,0,0,51,married
95456,1.181001e+09,punjab,lahore,urban,11810009,B,1,male,present,head of household,1950,0,0,56,married
95911,1.181001e+09,punjab,lahore,urban,11810014,B,2,female,present,wife/husband,1962,0,0,44,married
336702,3.032003e+09,nwfp,lower dir,rural,30320025,B,5,female,present,son/daughter,1996,0,0,10,unmarried
459249,4.132001e+09,balochistan,kharan,rural,41320006,B,4,male,present,son/daughter,1988,0,0,18,unmarried


In [6]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [7]:
# Fix NAs
C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].fillna(value=str('no'))
C_relevant['currently_enrolled'].isna().value_counts()

False    446425
Name: currently_enrolled, dtype: int64

In [8]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [9]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [10]:
# Select people in school enrollment age (I would say 4-15 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 10)]

In [11]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    40637
2    28356
3    22014
4    19711
Name: province, dtype: int64

In [12]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")
years06_07.loc[(years06_07["region"] == 1), 'region'] = 'urban'
years06_07.loc[(years06_07["region"] == 2), 'region'] = 'rural'

In [13]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
48,Sindh,Khairpur,20120
113,Balochistan,Bolan/kachhi,42620
13,Balochistan,Quetta,40111
105,Balochistan,Zhob,41820
14,Punjab,Attock,10120


In [14]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
106,Balochistan,Loralai,41920,4192
15,Punjab,Rawalpindi,10220,1022
45,Punjab,Bahawalpur,13220,1322
36,Punjab,Sahiwal,12320,1232
48,Sindh,Khairpur,20120,2012


In [15]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          92925
left_only         0
right_only        0
Name: _merge, dtype: int64

In [16]:
years06_07_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
46634,2102002102.0,13,9,male,unmarried,1,3.0,1.0,NaN,rural,2,2102,Sindh,Hyderabad,21020,both
80077,4042001310.0,5,7,male,unmarried,1,1.0,1.0,NaN,rural,4,4042,Balochistan,Chaghi,40420,both
80751,4052001404.0,11,4,male,unmarried,2,NaN,no,10.0,rural,4,4052,Balochistan,Sibbi,40520,both


In [17]:
# add year
years06_07_['year']=2005
years06_07_['marital_status']=years06_07_['marital_status'].astype('str')

#fix marital status categories
years06_07_.loc[(years06_07_['marital_status'] == "unmarried"), 'marital_status'] = 1
years06_07_.loc[(years06_07_['marital_status'] == "married"), 'marital_status'] = 2
years06_07_.loc[(years06_07_['marital_status'] == "only nikha has been solemanised,rukhsati not taken"), 'marital_status'] = 5
years06_07_.loc[(years06_07_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years06_07_.loc[(years06_07_['marital_status'] == "divorced"), 'marital_status'] = 4
years06_07_['marital_status']=years06_07_['marital_status'].astype('int')

# fix column names
years06_07_['province']=years06_07_['province_y']

In [18]:
#select relevant columns
years06_07_ = years06_07_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "subprovince",
        "year",
    ]
].copy()

In [19]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.csv', index_label="id")

# PART 2

In [20]:
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("yes", 1)
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("no", 0)
years06_07_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince,year
2029,1022002016.0,7,7,1,female,1,1.0,rural,Punjab,1022,Rawalpindi,2005
51817,2142002402.0,5,8,1,female,1,1.0,rural,Sindh,2142,MirPurKhas,2005
9991,1112003015.0,3,8,1,male,1,1.0,rural,Punjab,1112,T.T. Singh,2005
33190,1342003505.0,7,4,1,female,2,0.0,rural,Punjab,1342,Rahim Yar Khan,2005
45344,2101000910.0,4,6,1,male,1,1.0,urban,Sindh,2101,Hyderabad,2005


In [21]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer

# check region for anomalies

years06_07_["currently_enrolled"]= years06_07_["currently_enrolled"].astype('int').copy()
df_grp_1 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()
df_grp_2 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()

# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          408
left_only       0
right_only      0
Name: _merge, dtype: int64

In [22]:
df_grp_merge.isna().any()

sex                     False
subprovince             False
region                  False
currently_enrolled_x    False
currently_enrolled_y    False
_merge                  False
dtype: bool

In [23]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]

# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})

# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
260,female,Haripur,rural,192,154,0.802083
194,male,Thatta,rural,603,240,0.398010
321,female,Mandi Bahuddin,urban,0,0,NaN
19,male,Bhakhar,urban,0,0,NaN
390,female,Swat,rural,394,199,0.505076


In [24]:
df_grp_merge.to_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years06_07_merge_diff.csv")

In [25]:
df_grp_merge.isna().any()

sex                  False
subprovince          False
region               False
sample_population    False
enrolled_total       False
rate_enrollment       True
dtype: bool